In [ ]:
import os

os.environ["HF_HOME"] = "/home/yandex/APDL2425a/group_12/gorodissky/.cache/huggingface"
print(f"HF_HOME set to:\t\t {os.environ['HF_HOME']}")
import torch

print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.device_count())
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))


In [ ]:
from adais.src.post_processing import aggregators, run_eval_lib
import json
import os
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore', message='Labels passed were')

for temp in [0.4]:
    print(f"Running evaluation with temperature: {temp}")
    agg_temp = temp
    model_name = "Ministral-8B-Instruct-2410"
    dataset = "MMLU"
    version = "2026_01_18-22:17"

    aggregator_configs = [
        aggregators.AggregatorConfig(
            aggregator_type=aggregators.AggregatorType.SC,
            norm_type=aggregators.NormalizationType.NONE,
        ),
        aggregators.AggregatorConfig(
            aggregator_type=aggregators.AggregatorType.CISC,
            norm_type=aggregators.NormalizationType.SOFTMAX,
            temperature=agg_temp,  # This value is taken from the CISC paper.
            confidence_col_name="logit_confidence",  # P(True) in the paper.
        )
    ]

    PATH = f"/home/yandex/APDL2425a/group_12/gorodissky/google-research/cisc/output/{model_name}/confidence/{version}/{dataset}"
    with open(os.path.join(PATH, "experiment_conf.json"), "r") as f:
        expr_config = json.load(f)
    sample_sizes = range(1, 11)
    max_traces_for_diis = range(2, 24 ,2)

    stats = run_eval_lib.calculate_stats_for_model_and_dataset_path(
        expr_config["tag"],
        PATH,
        filter_answers=False,
        round_negative_conf_to_zero=False,
        re_compute_is_correct=False,
        aggregator_configs=aggregator_configs,
        traces_lens=sample_sizes,
        max_traces_for_diis=max_traces_for_diis,
        num_bootstrap=500,
        return_per_question_scores=False,
    ).score_stats

    print("="*60,"\nFinal results:")
    for metric_name, metric_scores in stats.items():
        print(f"{metric_name}: {metric_scores}")


    # Plotting

    # Accuracy vs avg number of traces

    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    mean_number_of_traces = stats["DIIS_mean_num_traces"]
    for metric_name, metric_scores in stats.items():
        if metric_name == "DIIS_mean_num_traces":
            continue
        if "CISC" in metric_name:
            if "DIIS" in metric_name:
                axes[0].plot( mean_number_of_traces, metric_scores, linestyle='--', marker='o', label=metric_name, color='orange')
            else:
                axes[0].plot(sample_sizes, metric_scores, label=metric_name, linestyle='-', marker='o', color='orange')
        else: # SC
            if "DIIS" in metric_name:
                axes[1].plot( mean_number_of_traces, metric_scores, linestyle='--', marker='o', label=metric_name, color='blue')
            else:
                axes[1].plot(sample_sizes, metric_scores, label=metric_name, linestyle='-', marker='o', color='blue')
                
    for i in range(2):
        axes[i].set_xlabel('Avg number of traces')
        axes[i].set_ylabel('Accuracy')
        axes[i].legend()
        axes[i].grid()
    plt.suptitle("Santiy: inverse scaling " + model_name + "-" + dataset + "-" + str(agg_temp))
    plt.tight_layout()
    plt.show()
    
    print("\n\n"+"="*60)